# Questões SQL
1 - Explique como estruturaria um data lake em três camadas:
*  Raw (dados brutos)
* Processed (dados tratados)
* Curated (dados prontos para análise)

Mostre um exemplo de diretório

---
**Camada RAW**

A camada RAW de um data lake é responsável por armazenar os dados da maneira mais próxima possível ao que foram recebidos originalmente, geralmente salvos em algum formato estruturado, como `csv`, `json`, `parquet`, `xml`.

Nesta camada não é esperado que haja nenhum tratamento dos dados, e isso inclui possíveis erros e duplicações nos dados.

Geralmente é esperado que esta camada possua acesso mais restrito, pois é nela que são feitas alguns tipos de atividades, como reprocesamento de dados e auditorias.

Neste projeto a camada RAW está representada na diretório `raw_data`, além de seus subdiretórios. Eles são o mais próximo das informações originais, o que é ilustrado pela quantidade de informações ausentes nas mais variadas tabelas e seus formatos.

**Camada Processed**

A camada processed, por sua vez, é a responsável por armazenar os dados após a limpeza, normalização e padronização.

Os dados são submetidos à processos de transformações como a remoção de duplicatas, conversão de tipos, padronização de nomes de colunas e enriquecimento a partir de outras fontes de dados.

Nesta etapa os dados são manipulados em formatos mais restritos, como `parquet` para aplicações de big data ou formatos estruturados mais simples, como `csv` e `json`.

São também estruturados, geralmente, por domínio de negócio e particionados por data.

**Camada Curated**

Esta é a camada final, planejada e otimizada para análise de dados.

É nela que é feita tarefas como a junção de tabelas, agregações e cálculos de indicadores.

Os formatos mais comuns são Parquet, Delta Lake ou outras proprietárias.

Um exemplo mais extenso de um data lake pode ser visualizado no seguinte esquema

```
/data_lake/
│
├── raw/
│   ├── vendas/
│   │   ├── api_vendas/2025/11/09/vendas_20251109_1200.json
│   │   └── api_vendas/2025/11/08/vendas_20251108_1200.json
│   ├── clientes/
│   │   └── upload_excel/2025/11/09/clientes_original.xlsx
│   └── logs/
│       └── app/2025/11/09/logs_20251109.txt
│
├── processed/
│   ├── vendas/
│   │   └── ano=2025/mes=11/dia=09/vendas.parquet
│   ├── clientes/
│   │   └── ano=2025/mes=11/dia=09/clientes_normalizados.parquet
│   └── produtos/
│       └── ano=2025/mes=11/dia=09/produtos_refinados.parquet
│
└── curated/
    ├── dashboards/
    │   └── vendas_por_estado.parquet
    ├── indicadores/
    │   ├── receita_mensal.parquet
    │   └── churn_rate.parquet
    └── modelos_ml/
        └── features_clientes.parquet

```
-----

**- Calcule o preço médio mensal por commodity, mostrando a variação percentual em relação ao mês anterior (função LAG)**

In [3]:
import pandas as pd
from sqlalchemy import create_engine, inspect, text
from dotenv import load_dotenv
import os

load_dotenv()

True

In [5]:
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT", 5432))
DB_NAME = os.getenv("DB_NAME")
SCHEMA = 'public'

In [9]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
insp = inspect(engine)

tables = insp.get_table_names(schema=SCHEMA)

In [25]:
for table in tables:
    cols = [col["name"] for col in insp.get_columns(table, schema=SCHEMA)]
    if "date" not in cols or not any(c for c in cols if "prazo" in c or "a_vista" in c):
        continue  # pula tabelas que não têm colunas de preço

    price_col = next(c for c in cols if "prazo" in c or "a_vista" in c)

    print(f"📊 Calculando variação mensal de '{price_col}' na tabela: {table}")

    query = f"""
    SELECT
        region_id,
        date_trunc('month', date) AS month,
        AVG("{price_col}") AS avg_price,
        LAG(AVG("{price_col}")) OVER (
            PARTITION BY region_id
            ORDER BY date_trunc('month', date)
        ) AS prev_avg_price,
        ROUND(
            (AVG("{price_col}") - LAG(AVG("{price_col}")) OVER (
                PARTITION BY region_id
                ORDER BY date_trunc('month', date)
            )) / NULLIF(LAG(AVG("{price_col}")) OVER (
                PARTITION BY region_id
                ORDER BY date_trunc('month', date)
            ), 0) * 100, 2
        ) AS var_percent
    FROM "{SCHEMA}"."{table}"
    GROUP BY region_id, date_trunc('month', date)
    ORDER BY region_id, month;
    """

    df = pd.read_sql(text(query), engine)
    display(df.tail())
    
    # Remova este break para processar todas as tabelas
    # O break está aqui apenas para evitar muitas saídas durante a exibição do exemplo
    break 

engine.dispose()


📊 Calculando variação mensal de 'a_vista_brl' na tabela: acucar_cristal_branco_cepea_esalq_sao_paulo


,region_id,month,avg_price,prev_avg_price,var_percent
266,1,2025-07-01 00:00:00-03:00,118.492174,126.437500,-6.28
267,1,2025-08-01 00:00:00-03:00,119.728571,118.492174,1.04
268,1,2025-09-01 00:00:00-03:00,118.647273,119.728571,-0.90
269,1,2025-10-01 00:00:00-03:00,114.956522,118.647273,-3.11
270,1,2025-11-01 00:00:00-03:00,109.294000,114.956522,-4.93


---
**- Liste os 5 produtos mais negociados no último ano**

É importante ressaltas que os dados atuais não permitem dizer quais são os 5 produtos mais negociados no ano por volume diretamente. Podemos, entretanto, usar *a quantidade de registros do ano*

In [32]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
insp = inspect(engine)

tables = insp.get_table_names(schema=SCHEMA)
results = []

for table in tables:
    cols = [c["name"] for c in insp.get_columns(table, schema=SCHEMA)]
    if "date" not in cols:
        continue  # ignora tabelas sem coluna de data

    query = text(f"""
        SELECT COUNT(*) AS registros_ultimo_ano
        FROM "{SCHEMA}"."{table}"
        WHERE date >= CURRENT_DATE - INTERVAL '1 year';
    """)

    with engine.connect() as conn:
        count = conn.execute(query).scalar()
        results.append({"produto": table, "registros_ultimo_ano": count})

# Criar DataFrame e ordenar
df = pd.DataFrame(results).sort_values(by="registros_ultimo_ano", ascending=False)

display(df.head(5))

,produto,registros_ultimo_ano
0,acucar_cristal_branco_cepea_esalq_sao_paulo,248
1,algodao_pluma_cepea_esalq_8_dias,248
2,indicador_acucar_cristal_santos_fob,248
3,indicador_arroz_casca_cepea_irga_rs,248
4,indicador_cafe_robusta_cepea_esalq,248


Novamente, não podemos chegar a conclusões profundas por falta de dados que dizem exatamente o que o questionamento deseja

---

**- Identifique registros anômalos (ex: preços negativos ou fora de faixa)**

Considerando valores em dólares (USD) para o cálculo de outliers em razão da estabilidade da moeda

In [45]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
insp = inspect(engine)

tables = insp.get_table_names(schema=SCHEMA)
anomalias = []

for table in tables:
    cols = [c["name"] for c in insp.get_columns(table, schema=SCHEMA)]
    price_cols = [c for c in cols if any(k in c.lower() for k in ["a_vista_usd", "prazo_de_8_dias_usd"])]

    if not price_cols or "date" not in cols:
        print('Nenhuma coluna de preço encontrada ou coluna de data ausente em', table)
        continue

    price_col = price_cols[0]
    print(f"Verificando anomalias em '{table}' ({price_col})...")

    # Carregar dados relevantes
    query = f'SELECT id, date, "{price_col}" AS price, region_id FROM "{SCHEMA}"."{table}" WHERE "{price_col}" IS NOT NULL'
    df = pd.read_sql(text(query), engine)

    if df.empty:
        continue

    # ---- 1️⃣ Preços negativos
    negativos = df[df["price"] < 0].copy()
    if not negativos.empty:
        negativos["tipo_anomalia"] = "preço_negativo"
        negativos['table'] = table
        anomalias.append(negativos)

    # ---- 2️⃣ Outliers (usando IQR)
    q1 = df["price"].quantile(0.25)
    q3 = df["price"].quantile(0.75)
    iqr = q3 - q1
    lim_inf = q1 - 1.5 * iqr
    lim_sup = q3 + 1.5 * iqr

    outliers = df[(df["price"] < lim_inf) | (df["price"] > lim_sup)].copy()
    if not outliers.empty:
        outliers["tipo_anomalia"] = "outlier_iqr"
        outliers['table'] = table
        anomalias.append(outliers)

print("\n✅ Análise concluída.")

# Consolidar anomalias de todas as tabelas
if anomalias:
    df_anomalias = pd.concat(anomalias, ignore_index=True)
    print(f"\nTotal de anomalias encontradas: {len(df_anomalias)}")
    display(df_anomalias.head(10))
else:
    print("Nenhuma anomalia detectada.")

engine.dispose()

Verificando anomalias em 'acucar_cristal_branco_cepea_esalq_sao_paulo' (a_vista_usd)...
Verificando anomalias em 'algodao_pluma_cepea_esalq_8_dias' (prazo_de_8_dias_usd)...
Verificando anomalias em 'indicador_acucar_cristal_santos_fob' (a_vista_usd)...
Verificando anomalias em 'indicador_arroz_casca_cepea_irga_rs' (a_vista_usd)...
Verificando anomalias em 'indicador_cafe_robusta_cepea_esalq' (a_vista_usd)...
Verificando anomalias em 'indicador_cafe_arabica_cepea_esalq' (a_vista_usd)...
Verificando anomalias em 'indicador_semanal_etanol_anidro_cepea_esalq_sao_paulo' (a_vista_usd)...
Verificando anomalias em 'indicador_semanal_etanol_hidratado_outros_fins_cepea_esalq_s' (a_vista_usd)...
Verificando anomalias em 'indicador_semanal_etanol_hidratado_combustivel_cepea_esalq_s' (a_vista_usd)...
Verificando anomalias em 'indicador_soja_cepea_esalq_parana' (a_vista_usd)...
Verificando anomalias em 'indicador_soja_cepea_esalq_paranagua' (a_vista_usd)...
Verificando anomalias em 'preco_medio_trig

,id,date,price,region_id,tipo_anomalia,table
0,1832,2010-10-07,41.65,1,outlier_iqr,acucar_cristal_branco_cepea_esalq_sao_paulo
1,1833,2010-10-08,42.82,1,outlier_iqr,acucar_cristal_branco_cepea_esalq_sao_paulo
2,1834,2010-10-11,42.85,1,outlier_iqr,acucar_cristal_branco_cepea_esalq_sao_paulo
3,1835,2010-10-13,43.28,1,outlier_iqr,acucar_cristal_branco_cepea_esalq_sao_paulo
4,1836,2010-10-14,43.08,1,outlier_iqr,acucar_cristal_branco_cepea_esalq_sao_paulo
5,1837,2010-10-15,43.21,1,outlier_iqr,acucar_cristal_branco_cepea_esalq_sao_paulo
6,1838,2010-10-18,43.46,1,outlier_iqr,acucar_cristal_branco_cepea_esalq_sao_paulo
7,1839,2010-10-19,42.91,1,outlier_iqr,acucar_cristal_branco_cepea_esalq_sao_paulo
8,1840,2010-10-20,43.27,1,outlier_iqr,acucar_cristal_branco_cepea_esalq_sao_paulo
9,1841,2010-10-21,42.86,1,outlier_iqr,acucar_cristal_branco_cepea_esalq_sao_paulo
